<a href="https://www.kaggle.com/code/scottxchoo/cactus-3-improve-performance-submission?scriptVersionId=145052895" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Improve Performance & Submission

In Baseline, we used a simple CNN model. This time, we'll make four improvements to imporve performance.

1. Perform different image transformations
2. Create a deeper CNN model.
3. Use a better optimizer.
4. Increase the number of epochs in training.

Except for these four things, the code is similar to the baseline.

[베이스라인](https://www.kaggle.com/code/scottxchoo/cactus-2-baseline-model)에서는 간단한 CNN 모델을 사용했습니다. 이번에는 다음 네 가지를 개선해 성능을 높여보겠습니다.

1. 다양한 이미지 변환을 수행합니다.
2. 더 깊은 CNN 모델을 만듭니다.
3. 더 뛰어난 옵티마이저를 사용합니다.
4. 훈련 시 에폭 수를 늘립니다.

이상의 네 가지를 제외하고는 [베이스라인](https://www.kaggle.com/code/scottxchoo/cactus-2-baseline-model)과 코드가 비슷합니다.

먼저, 시드값 고정부터 'Prepare Data (데이터 준비)'의 '[2] Define dataset classes (데이터셋 클래스 정의)'까지 베이스라인과 동일하게 진행합니다.

## 1. Fix the seed value and set up your GPU equipment (시드값 고정 및 GPU 장비 설정)

### [1] Fix the seed value (시드값 고정)

In [ ]:
import torch # 파이토치
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)                # 파이썬 난수 생성기 시드 고정
np.random.seed(seed)             # 넘파이 난수 생성기 시드 고정
torch.manual_seed(seed)          # 파이토치 난수 생성기 시드 고정 (CPU 사용 시)
torch.cuda.manual_seed(seed)     # 파이토치 난수 생성기 시드 고정 (GPU 사용 시)
torch.cuda.manual_seed_all(seed) # 파이토치 난수 생성기 시드 고정 (멀티GPU 사용 시)
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False    # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False      # cudnn 사용 해제

### [2] Set up GPU equipment (GPU 장비 설정)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
device

## 2. Prepare Data (데이터 준비)

In [ ]:
import pandas as pd

data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압축 풀기
with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
# 테스트 이미지 데이터 압축 풀기
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

### [1] Separate training/validation data (훈련/검증 데이터 분리)

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(labels,
                                test_size = 0.1, # (1)
                                stratify = labels['has_cactus'], # (2)
                                random_state = 50)

In [ ]:
print('훈련 데이터 개수:', len(train))
print('검증 데이터 개수:', len(valid))

### [2] Define dataset classes (데이터셋 클래스 정의)

In [ ]:
import cv2 # OpenCV 라이브러리
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

In [ ]:
class ImageDataset(Dataset):
    # (1) 초기화 메서드(생성자)
    def __init__(self, df, img_dir = './', transform = None):
        super().__init__() ## (2) 상속받은 Dataset의 생성자 호출
        # (3) 전달받은 인수들 저장
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # (4) 데이터셋 크기 반환 메서드
    def __len__(self):
        return len(self.df)
    
    # (5) 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]                  # 이미지 ID
        img_path = self.img_dir + img_id               # (6) 이미지 파일 경로
        image = cv2.imread(img_path)                   # 이미지 파일 읽기 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        label = self.df.iloc[idx, 1]                   # 이미지 레이블(타깃값)
        
        if self.transform is not None:
            image = self.transform(image) # (7) 변환기가 있다면 이미지 변환
        return image, label # (8)

### [3] Image transformation and data augmentation (이미지 변환과 데이터 증강)

We mentioned earlier that you can apply image transforms when creating a dataset with the ImageDataset class. Examples of popular image transforms include edge removal, symmetry, rotation, resize, original, noise, color change, block, and blur.

Why do we transform images? The answer is simple: to generate more data. You'll be able to create 8 or more images from a single image. Deep learning models are generally more accurate the more training data they have, so more meaningful data is better. This practice of converting images to increase the number of data is called data augmentation.

앞서 ImageDataset 클래스로 데이터셋을 만들 때 이미지 변환기를 적용할 수 있다고 했습니다. 많이 쓰이는 이미지 변환의 예로는 가장자리 제거, 대칭, 회전, 크기 변경, 원본, 노이즈, 색상 변경, 차단, 흐릿하게 등이 있습니다.

이미지를 변환하는 이유는 무엇일까요? 바로 데이터를 더 많이 생성하기 위해서입니다. 하나의 이미지로 8개 넘는 이미지를 만들 수 있게 됩니다. 딥러닝 모델은 대체로 훈련 데이터가 많을수록 정확해지므로 의미 있는 데이터는 많을수록 좋습니다. 이렇게 이미지를 변환하여 데이터 수를 늘리는 방식을 데이터 증강(data augmentation)이라고 합니다.

### [4] Define image transforms (이미지 변환기 정의)

We'll define our own image transforms to augment our data. To improve performance, we'll utilize different image transforms, creating one for training data and another for validation and testing data. This is because it's good to adapt the model to different situations during training, but during evaluation and testing, it can become unpredictable if it deviates too much from the original image.

데이터를 증강해줄 이미지 변환기를 직접 정의해보죠. 성능을 개선하기 위해 다양한 이미지 변환기를 활용할 텐데, 훈련 데이터용과 검증 및 테스트 데이터용을 따로 만듭니다. 훈련 시에는 모델을 다양한 상황에 적응시키는 게 좋지만, 평가 및 테스트 시에는 원본 이미지와 너무 달라지면 예측하기 어려워질 수 있기 때문입니다.

In [ ]:
from torchvision import transforms # 이미지 변환을 위한 모듈

# 훈련 데이터용 변환기
transform_train = transforms.Compose([transforms.ToTensor(), # (1)
                                     transforms.Pad(32, padding_mode = 'symmetric'), # (2)
                                     transforms.RandomHorizontalFlip(), # (3)
                                     transforms.RandomVerticalFlip(), # (4)
                                     transforms.RandomRotation(10), # (5)
                                     transforms.Normalize((0.485, 0.456, 0.406), # (6)
                                                          (0.229, 0.224, 0.225))])

# 검증 및 테스트 데이터용 변환기
transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Pad(32, padding_mode = 'symmetric'),
                                     transforms.Normalize((0.485, 0.456, 0.406),
                                                          (0.229, 0.224, 0.225))])

We have combined multiple transforms into one with transforms.Compose().

(6) transforms.Normalize() : Normalizes the data to the specified mean and variance. You can set them to values between 0 and 1. Here, I normalized the mean to (0.485, 0.456, 0.406) and the variance to (0.229, 0.224, 0.225).

Two questions arise here.

First, why do we have three means and three variances?
The colors in the image data are red (R), green (G), and blue (B). We need to normalize red, green, and blue separately, which is why the mean and variance have three values.

Second, so why is the mean (0.485, 0.456, 0.406) and the variance (0.229, 0.224, 0.225)?
Other values are fine, but when dealing with images, we usually normalize to these values. These values come from data from ImageNet, which has over a million images. I could calculate the mean and variance directly from the images I'm using, but it's a hassle, so I usually just use these values.

transforms.Compose()로 여러 변환기를 하나로 묶었습니다.

(6) transforms.Normalize() : 데이터를 지정한 평균과 분산에 맞게 정규화해줍니다. 0~1 사이 값으로 설정해주면 되는데, 여기서는 평균을 (0.485, 0.456, 0.406)으로, 분산을 (0.229, 0.224, 0.225)로 정규화했습니다.

여기서 두 가지 의문이 듭니다.

첫째, 왜 평균과 분산이 각각 세 개씩 있을까요?
이미지 데이터의 색상읜 빨강(R), 초록(G), 파랑(B)으로 구성돼 있습니다. 빨강, 초록, 파랑을 각각 정규화해야 해서 평균과 분선에 값을 세 개씩 전달한 겁니다.

둘째, 그렇다면 왜 평균은 (0.485, 0.456, 0.406)이고 분산은 (0.229, 0.224, 0.225)일까요?
다른 값을 해도 상관없지만 이미지를 다룰 때는 보통 이 값들로 정규화합니다. 이 값들은 백만 개 이상의 이미지를 보유한 이미지넷(ImageNet)의 데이터로부터 얻은 값입니다. 내가 사용할 이미지들로부터 평균과 분산을 직접 구해도 되지만 번거롭기 때문에 대개 이 값을 그대로 사용합니다.

### [5] Create datasets and data loaders (데이터셋 및 데이터 로더 생성)

Create training and validation datasets with the ImageDataset class. This is exactly the same as the baseline code, except for the transforms you pass in: when you create a training dataset, you pass in a transform for training, and when you create a validation dataset, you pass in a transform for validation/testing.

ImageDataset 클래스로 훈련 및 검증 데이터셋을 만듭니다. 전달하는 변환기를 제외하면 베이스라인 코드와 똑같습니다. 훈련 데이터셋을 만들 때는 훈련용 변환기를, 검증 데이터셋을 만들 때는 검증/테스트용 변환기를 전달합니다.

In [ ]:
dataset_train = ImageDataset(df = train, img_dir = 'train/',
                             transform = transform_train)
dataset_valid = ImageDataset(df = valid, img_dir = 'train/',
                             transform = transform_test)

In [ ]:
from torch.utils.data import DataLoader # 데이터 로더 클래스

loader_train = DataLoader(dataset = dataset_train, batch_size = 32, shuffle = True)
loader_valid = DataLoader(dataset = dataset_valid, batch_size = 32, shuffle = False)

When we created our dataset earlier, we passed in an image transform, which will perform an image transformation each time we load the data into the data loader. In this case, the transformers RandomHorizontalFlip(), RandomVerticalFlip(), and RandomRotation() will transform differently each time because they randomize the transformations. This means that the original image is the same, but you get the effect of training with different images in the first and second epochs. This is the "data augmentation" technique.

앞서 데이터셋을 만들 때 이미지 변환기를 전달했습니다. 그러면 데이터 로더로 데이터를 불러올 때마다 이미지 변환을 수행합니다. 이때 변환기 중 RandomHorizontalFlip(), RandomVerticalFlip(), RandomRotation()은 변환을 무작위로 가하기 때문에 매번 다르게 변환합니다. 즉, 원본 이미지는 같지만 첫 번째 에폭과 두 번째 에폭에서 서로 다른 이미지로 훈련하는 효과를 얻을 수 있는 거죠. 이것이 바로 '데이터 증강' 기법입니다.

## 3. Create Model (모델 생성)

Now that we have our data, let's design our CNN model. The baseline has two convolutional and two max-pooling layers, followed by a mean-pooling layer and a fully-connected layer.

This time, we're going to build a deeper CNN. Deeper neural network layers usually lead to better predictive power. However, be careful not to go too deep as it can lead to overfitting. We'll also apply batch normalization and change the activation function to Leaky ReLU for better performance.

We will also have a total of five layers {convolutional, batch normalization, max pooling} and two fully-connected layers.

We'll utilize `nn.Sequential()` to design the neural network layers.

데이터가 준비되었으니 이제 CNN 모델을 설계해봅시다. 베이스라인에는 합성곱과 최대 풀링 계층이 두 개씩이고, 이어서 평균 플링 계층과 전결합 계층이 하나씩 있습니다.

이번에는 더 깊은 CNN을 만들겠습니다. 신경망 계층이 깊어지면 대체로 예측력이 좋아집니다. 다만 지나치게 깊으면 과대적합될 우려가 있으니 유의하세요. 아울러 배치 정규화를 적용하고 활성화 함수를 Leaky ReLU로 바꿔서 성능을 더 높여보겠습니다.

또한, {합성곱, 배치 정규화, 최대 풀링} 계층이 총 다섯 개에 전결합 계층도 두 개로 늘릴 것입니다.

`nn.Sequential()`을 활용해 신경망 계층을 설계하겠습니다.

In [ ]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

class Model(nn.Module):
    # 신경망 계층 정의
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
        # 1~5번째 {합성곱, 배치 정규화, 최대 풀링} 계층
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels = 3, out_channels = 32,
                                              kernel_size = 3, padding = 2),
                                    nn.BatchNorm2d(32), # (1) 배치 정규화
                                    nn.LeakyReLU(), # (2) LeakyReLu 활성화 함수
                                    nn.MaxPool2d(kernel_size = 2))

        self.layer2 = nn.Sequential(nn.Conv2d(in_channels = 32, out_channels = 64,
                                              kernel_size = 3, padding = 2),
                                    nn.BatchNorm2d(64), # (1) 배치 정규화
                                    nn.LeakyReLU(), # (2) LeakyReLu 활성화 함수
                                    nn.MaxPool2d(kernel_size = 2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels = 64, out_channels = 128,
                                              kernel_size = 3, padding = 2),
                                    nn.BatchNorm2d(128), # (1) 배치 정규화
                                    nn.LeakyReLU(), # (2) LeakyReLu 활성화 함수
                                    nn.MaxPool2d(kernel_size = 2))
        
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels = 128, out_channels = 256,
                                              kernel_size = 3, padding = 2),
                                    nn.BatchNorm2d(256), # (1) 배치 정규화
                                    nn.LeakyReLU(), # (2) LeakyReLu 활성화 함수
                                    nn.MaxPool2d(kernel_size = 2))
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels = 256, out_channels = 512,
                                              kernel_size = 3, padding = 2),
                                    nn.BatchNorm2d(512), # (1) 배치 정규화
                                    nn.LeakyReLU(), # (2) LeakyReLu 활성화 함수
                                    nn.MaxPool2d(kernel_size = 2))
        
        # 평균 풀링 계층
        self.avg_pool = nn.AvgPool2d(kernel_size = 4)
        # 전결합 계층
        self.fc1 = nn.Linear(in_features = 512 * 1 * 1, out_features = 64)
        self.fc2 = nn.Linear(in_features = 64, out_features = 2)
    
    # 순전파 출력 정의
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avg_pool(x)
        x = x.view(-1, 512 * 1 * 1) # 평탄화
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
model = Model().to(device)
model

## 4. Train Model (모델 훈련)

Just like in the baseline, we'll set up our loss function and optimizer and then start training.

베이스라인 때와 마찬가지로 손실 함수와 옵티마이저를 설정한 후 훈련에 돌입하겠습니다.

### Set Loss Function & Optimizer (손실 함수와 옵티마이저 설정)

The loss function will be CrossEntropyLoss(), the same as in the baseline.

손실 함수는 베이스라인과 동일하게 CrossEntropyLoss()로 하겠습니다.

In [ ]:
# 손실 함수
criterion = nn.CrossEntropyLoss()

Let's change the optimizer to Adamax (in the baseline, we used SGD, the default optimizer). You can think of Adamax as an improved version of Adam. Of course, Adamax doesn't always guarantee better results than SGD or Adam, and it's hard to tell which optimizer is better until you actually test it, so it's best to experiment.

옵티마이저는 Adamax로 바꿔보겠습니다(베이스라인에서는 기본 옵티마이저인 SGD를 사용했습니다). Adamax는 Adam의 개선 버전이라고 보면 됩니다. 물론 Adamax가 SGD나 Adam보다 항상 나은 결과를 보장하는 건 아닙니다. 게다가 실제로 테스트해보기 전까진 어떤 옵티마이저가 더 좋은지 판단하기가 쉽지 않으니 여러 차례 실험을 해보는 게 좋습니다.

In [ ]:
# 옵티마이저
optimizer = torch.optim.Adamax(model.parameters(), lr = 0.00006)

### Train Model (모델 훈련)

To train more, we'll increase the number of epochs from 10 to 70, since we have more data to train with. The rest of the code is the same as the baseline.

훈련을 더 많이 하기 위해 에폭 수를 10에서 70으로 늘리겠습니다. 데이터를 증강시켜 훈련할 데이터가 많아졌으니 에폭을 더 늘려도 되기 때문입니다. 나머지 코드는 베이스라인과 동일합니다.

In [ ]:
epochs = 70 # 총 에폭
# 총 에폭만큼 반복
for epoch in range(epochs):
    epoch_loss = 0 # 에폭별 손실값 초기화
    
    # '반복 횟수'만큼 반복
    for images, labels in loader_train:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가
        epoch_loss += loss.item()
        # 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step()
        
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch + 1}/{epochs}] - 손실값 : {epoch_loss/len(loader_train):.4f}')

## 5. Validate Performance (성능 검증)

Let's evaluate the model performance with validation data. Again, the code is no different from the baseline.

검증 데이터로 모델 성능을 평가해보겠습니다. 역시 코드는 베이스라인과 다를 바 없습니다.

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수 임포트
# 실젯값과 예측 확률값을 담을 리스트 초기화
true_list = []
preds_list = []

model.eval() # 모델을 평가 상태로 설정

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in loader_valid:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당
        images = images.to(device)
        labels = labels.to(device)
        
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim = 1)[:, 1] # 예측 확률
        true = labels.cpu() # 실젯값
        # 예측 확률과 실젯값을 리스트에 추가
        preds_list.extend(preds)
        true_list.extend(true)
        
# 검증 데이터 ROC AUC 점수 계산
print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')

The baseline ROC AUC was 0.9900, but the refinements worked and it's now 0.9998. The maximum value of ROC AUC is 1, which is a near-perfect score. This means that we have almost perfect classification of the validation data.

베이스라인의 ROC AUC는 0.9900이었는데, 개선 작업이 효과를 발휘해 0.9998이 되었네요. ROC AUC의 최댓값이 1이니 거의 완벽에 가까운 점수입니다. 검증 데이터를 거의 완벽히 분류해냈다는 뜻입니다.

## 6. Predict & Submission (예측 및 제출)

Now it's time to make predictions with our test data. Again, we used the transform_test transformer to create our dataset.

이제 테스트 데이터로 예측해봐야겠죠? 이번에도 transform_test 변환기를 이용해 데이터셋을 만들었습니다.

In [ ]:
# 데이터셋과 데이터 로더 생성
dataset_test = ImageDataset(df = submission, img_dir = 'test/',
                            transform = transform_test)
loader_test = DataLoader(dataset = dataset_test, batch_size = 32, shuffle = False)

# 예측 수행
model.eval() # 모델을 평가 상태로 설정

preds = [] # 타깃 예측값 저장용 변수 초기화

with torch.no_grad(): # 기울기 계산 비활성화
    for images, _ in loader_test:
        # 이미지 데이터 미니배치를 장비에 할당
        images = images.to(device)
        
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 타깃값이 1일 확률(예측값)
        preds_part = torch.softmax(outputs.cpu(), dim = 1)[:, 1].tolist()
        # preds에 preds_part 이어붙이기
        preds.extend(preds_part)

Create a submission file and delete any directories you no longer need.

제출 파일을 만들고 이제 필요 없는 디렉터리는 지워줍니다.

In [ ]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index = False)

import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')

The End.